In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as web

/Users/berkatbhatti/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

### lets go back a few years for the data

In [3]:
data = pd.read_csv("resources/final_sandp.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Symbol  478 non-null    object
 1   Name    478 non-null    object
 2   Sector  478 non-null    object
dtypes: object(3)
memory usage: 11.3+ KB


In [5]:
tickers = list(data["Symbol"])

In [6]:
tickers.append("^GSPC")

In [7]:
workingData = pd.DataFrame()
for t in tickers:
    workingData[t] = web.DataReader(t, data_source="yahoo", start = "2016-1-1")["Adj Close"]

RemoteDataError: No data fetched for symbol SCG using YahooDailyReader

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(workingData.isnull(), yticklabels=False, cbar=False)

In [ ]:
final_Data = workingData.dropna(axis = 1)

In [ ]:
final_Data.info()

In [ ]:
daily_returns = pd.DataFrame()
for t in tickers:
    daily_returns[f"{t} daily returns"] = workingData[f"{t}"].pct_change()

In [ ]:
sns.heatmap(daily_returns.isnull())

In [ ]:
final_returns = daily_returns.dropna()

In [ ]:
sns.heatmap(final_returns.isnull(),yticklabels=False, cbar=False)

In [ ]:
returns_Corr = final_returns.corr()
returns_Corr

In [ ]:
corr_to_SandP = returns_Corr.corrwith(returns_Corr["^GSPC daily returns"])

In [ ]:
corr_to_SandP.sort_values(ascending =False).head(60)

In [ ]:
final_returns.head()

In [ ]:
sns.heatmap(final_returns.isnull())

In [ ]:
daily_returns

#### ets check the volatility of each secutiry

In [ ]:
del vol

In [ ]:
vol = []
for x in final_returns.columns:
    vol.append(final_returns[f"{x}"].std() * np.sqrt(252))

In [ ]:
len(vol)

In [ ]:
mean_daily = []
for x in final_returns.columns:
    mean_daily.append(final_returns[f"{x}"].mean() * 252)

In [ ]:
mean_daily[0:10]

In [ ]:
ss = pd.read_csv("resources/final_sandp.csv")

In [ ]:
ss

In [ ]:
symbol_to_company = dict(list(zip(ss["Symbol"], ss["Name"])))

In [ ]:
symbol_to_company["^GSPC"] = "Sandp500"

In [ ]:
symbol_to_company

In [ ]:
ticker_series = pd.Series(workingData.columns)

In [ ]:
volatility_series = pd.Series(vol)

In [ ]:
av_return_Series = pd.Series(mean_daily)

In [ ]:
main_data = pd.concat([ticker_series, volatility_series,av_return_Series],axis=1)
main_data.columns = ["symbol", "vol", "avg daily return"]

In [ ]:
main_data["company"] = main_data["symbol"].apply(lambda x: symbol_to_company[x])

In [ ]:
symbol_to_sector = dict(list(zip(ss["Symbol"], ss["Sector"])))

In [ ]:
symbol_to_sector["^GSPC"] = "Index"

In [ ]:
main_data["sector"] = main_data["symbol"].apply(lambda x: symbol_to_sector[x])

In [ ]:
main_data.sort_values("vol", ascending = False)

In [ ]:
main_data.to_csv("resources/maindata.csv")